<a href="https://colab.research.google.com/github/ashrafsusts19/machine_learning_lab/blob/main/task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing
from google.colab import drive
drive.mount('/content/drive')

pathPneu = '/content/drive/MyDrive/X-Ray/train/PNEUMONIA'
pathNormal = '/content/drive/MyDrive/X-Ray/train/NORMAL'
pathValPneu = '/content/drive/MyDrive/X-Ray/val/PNEUMONIA'
pathValNormal = '/content/drive/MyDrive/X-Ray/val/NORMAL'
pathTestPneu = '/content/drive/MyDrive/X-Ray/test/PNEUMONIA'
pathTestNormal = '/content/drive/MyDrive/X-Ray/test/NORMAL'

normalFiles = os.listdir(pathNormal)
pneuFiles = os.listdir(pathPneu)
normalValFiles = os.listdir(pathValNormal)
pneuValFiles = os.listdir(pathValPneu)
normalTestFiles = os.listdir(pathTestNormal)
pneuTestFiles = os.listdir(pathTestPneu)


data = []
countNormal, countPneu = 0, 0
countTestNormal, countTestPneu = 0, 0

for img in normalFiles:
    countNormal += 1
    image=Image.open(pathNormal +'/'+img).resize((128,128)).convert('RGB')
    image=np.array(image)
    data.append(image)
    # if countNormal == 25:
    #   break

for img in pneuFiles:
    countPneu += 1
    image=Image.open(pathPneu + '/' +img).resize((128,128)).convert('RGB')
    image=np.array(image)
    data.append(image)
    # if countPneu == 25:
    #   break

dataVal = []
for img in normalValFiles:
    image=Image.open(pathValNormal +'/'+img).resize((128,128)).convert('RGB')
    image=np.array(image)
    dataVal.append(image)


for img in pneuValFiles:
    image=Image.open(pathValPneu +'/'+img).resize((128,128)).convert('RGB')
    image=np.array(image)
    dataVal.append(image)


dataTest = []
for img in normalTestFiles:
    countTestNormal += 1
    image=Image.open(pathTestNormal +'/'+img).resize((128,128)).convert('RGB')
    image=np.array(image)
    dataTest.append(image)
    # if countTestNormal == 25:
    #     break


for img in pneuTestFiles:
    countTestPneu += 1
    image=Image.open(pathTestPneu +'/'+img).resize((128,128)).convert('RGB')
    image=np.array(image)
    dataTest.append(image)
    # if countTestPneu == 25:
    #     break


print("Number of Samples:", len(data))
print("Normal Sample:", len(normalFiles))
print("Pneumonia Samples:", len(pneuFiles))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of Samples: 5216
Normal Sample: 1341
Pneumonia Samples: 3875


In [47]:
allSampleResults = [0] * countNormal + [1] * countPneu
allValSampleResults = [0] * len(normalValFiles) + [1] * len(pneuValFiles)
allTestSampleResults = [0] * countTestNormal + [1] * countTestPneu

X = np.array(data)
Y = np.array(allSampleResults)

X_val = np.array(dataVal)
Y_val = np.array(allValSampleResults)

X_Test = np.array(dataTest)
Y_Test = np.array(allTestSampleResults)

X_scaled = X / 255
X_val_scaled = X_val / 255
X_test_scaled = X_Test / 255

In [ ]:
noClasses = 2

model = keras.Sequential()

model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(128, kernel_size=(4, 4), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(3, 3)))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Dense(noClasses, activation = 'sigmoid'))

model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)


history = model.fit(X_scaled,Y, validation_data = (X_val_scaled, Y_val), verbose = 1, epochs = 5)

Epoch 1/5
163/163 [==============================] - 581s 4s/step - loss: 0.2951 - accuracy: 0.8834 - val_loss: 0.7412 - val_accuracy: 0.7500
Epoch 2/5
163/163 [==============================] - 579s 4s/step - loss: 0.1331 - accuracy: 0.9534 - val_loss: 0.2888 - val_accuracy: 0.9375
Epoch 3/5
163/163 [==============================] - 581s 4s/step - loss: 0.1008 - accuracy: 0.9632 - val_loss: 0.2831 - val_accuracy: 0.9375
Epoch 4/5
163/163 [==============================] - 588s 4s/step - loss: 0.0780 - accuracy: 0.9741 - val_loss: 0.4313 - val_accuracy: 0.8750
Epoch 5/5
 88/163 [===============>..............] - ETA: 4:18 - loss: 0.0855 - accuracy: 0.9702

In [49]:
loss, accuracy = model.evaluate(X_test_scaled, Y_Test)
print("Loss:", loss, "Accuracy:", accuracy)


20/20 [==============================] - 18s 875ms/step - loss: 1.4314 - accuracy: 0.7035
Loss: 1.4313623905181885 Accuracy: 0.7035256624221802
